#  Evaluation Demo

### Imports

In [1]:
import io
import base64
import glob
import os

import gzip
import json
import numpy as np
import torch
from IPython.display import HTML
from skvideo.io import FFmpegReader, ffprobe, vwrite
from torch.autograd import Variable
from ptcap.trainers import DataParallelWrapper
from ptcap.scores import ( caption_accuracy, first_token_accuracy, token_accuracy)
from ptcap.data.annotation_parser import JsonParser
from collections import OrderedDict
from collections import Counter, namedtuple



/home/farzaneh/venv/ptcap/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)


In [2]:
-

In [3]:
stop_words = ['A', 'AN', 'THE', '<END>']

def safe_div(x,y):
    if y == 0:
        return 0
    return x / y
def fscore(precision, recall, beta=1):
    numerator = (1.0 + (beta ** 2)) * precision * recall
    denominator = ((beta ** 2) * precision) + recall
    return {"fscore": safe_div(numerator, denominator)}
class LCS(object):
    """
    The main functionality of this class is to compute the LCS (Lowest Common
    Subsequence) between a caption and prediction. By default, it returns the
    precision and recall values calculated based on the LCS between a prediction
    and a caption.
    """
    def __init__(self, functions_list, tokenizer):
        """
        Initializes functions_list and tokenizer.
        Args:
        functions_list: A list of the functions that will be applied on the
        precision and recall values calculated based on the LCS between a
        prediction and a caption.
        """

        self.functions_list = functions_list
        self.scores_container = OrderedDict()
        self.scores_dict = OrderedDict()
        self.tokenizer = tokenizer

    def __call__(self, outputs):
        string_predictions = [self.tokenizer.get_string(str_pred.data.numpy())
                              for str_pred in outputs.predictions]
        return self.score_batch(string_predictions, outputs.string_captions)

    def collect_scores(self, batch_scores_dict, scores_dict):
        for metric, metric_value in scores_dict.items():
            if metric not in batch_scores_dict:
                batch_scores_dict[metric] = [metric_value]
            else:
                batch_scores_dict[metric].append(metric_value)
        return batch_scores_dict

    @classmethod
    def compute_lcs(cls, prediction, caption):
        num_rows = len(prediction)
        num_cols = len(caption)

        table = [[0] * (num_cols + 1) for _ in range(num_rows + 1)]
        for i in range(1, num_rows + 1):
            for j in range(1, num_cols + 1):
                if prediction[i - 1] == caption[j - 1]:
                    table[i][j] = table[i - 1][j - 1] + 1
                else:
                    table[i][j] = max(table[i][j - 1], table[i - 1][j])
        return table, table[num_rows][num_cols]

    def mean_scores(self, batch_scores_dict):
        for metric, metric_value in batch_scores_dict.items():
            batch_scores_dict[metric] = np.mean(metric_value)
        return batch_scores_dict

    def score_batch(self, predictions, captions):
        assert len(predictions) == len(captions)

        batch_scores_dict = OrderedDict()
        for count, (prediction, caption) in enumerate(zip(predictions,
                                                          captions)):
            scores_dict = self.score_sample(prediction.split(), caption.split())
            batch_scores_dict = self.collect_scores(batch_scores_dict,
                                                    scores_dict)

        batch_scores_dict = self.mean_scores(batch_scores_dict)
        return batch_scores_dict

    def score_sample(self, prediction, caption):
        scores_dict = OrderedDict()
        _, lcs_score = self.compute_lcs(prediction, caption)
        scores_dict["precision"] = safe_div(lcs_score, len(prediction))
        scores_dict["recall"] = safe_div(lcs_score, len(caption))

        for score_function in self.functions_list:
            scores_dict.update(score_function(scores_dict["precision"],
                                              scores_dict["recall"]))

        return scores_dict

### Tool to deal with mpeg videos

In [4]:
def show_video(video_filenames):
    """
    Tool to display videos inside the notebook.
    """
    
    if type(video_filenames) is not list:
        video_filenames = [video_filenames]
    
    html_code = ''
    for filename in video_filenames:
        video = io.open(filename, 'r+b').read()
        encoded = base64.b64encode(video)
        html_code += '''
        <video alt="test" width="640" height="480" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        '''.format(encoded.decode('ascii'))
        
    return HTML(data= html_code)


def open_mpeg_video(fname, framerate, size):
    """
    Open an mpeg video, and return it as a numpy array.
    """
    
    metadata = ffprobe(fname)
    duration = float(metadata['video']['@duration'])
    # Compute corresponding nb of frames
    nframes = int(duration * framerate)
    oargs = {
        "-r": "%d" % framerate,
        "-vframes": "%d" % nframes,
        "-s": "%dx%d" % (size[0], size[1])
    }
    # Open file
    reader = FFmpegReader(fname, inputdict={}, outputdict=oargs)
    video = []
    # Get frames until there is no more
    for frame in reader.nextFrame():
        video.append(frame)
    # Return as a numpy array
    return np.array(video)

## A) Model Instantiation

In [5]:
from ptcap.model.captioners import EncoderDecoder
from ptcap.model.encoders import CNN3dLSTMEncoder
from ptcap.model.pretrained_encoders import FCEncoder, JesterEncoder, BIJesterEncoder
from ptcap.model.decoders import LSTMDecoder, CoupledLSTMDecoder
  
#net = FullyConvolutionalNet(num_classes=178)jester1024_cutoff_300_ssssssss/

net = EncoderDecoder(
        encoder=BIJesterEncoder,
        decoder=CoupledLSTMDecoder,
        encoder_kwargs={"freeze": False},#, "pretrained_path": "/home/farzaneh/PycharmProjects/pretrained_nets/fully_conv_net_on_smtsmt_20170627/model.checkpoint"},
        decoder_kwargs={"embedding_size": 256, "hidden_size": 1024, "num_lstm_layers": 2, 
        "vocab_size": 2986, "num_step" :17}, 
        gpus=[0]).cuda()
net = DataParallelWrapper(net, device_ids=[0]).cuda(0)

gpus: [0]


## B) Load weights

In [6]:
path = '/home/farzaneh/PycharmProjects/pytorch-captioning/results/clapnet_balanced_tokens/'
# path = '/home/farzaneh/PycharmProjects/pytorch-captioning/results/clapnet_captioning_only_f0.1'

checkpoint = torch.load(path + '/model.best')


net.load_state_dict(checkpoint["model"])


## C) Load Tokenizer

In [7]:
from ptcap.data.tokenizer import Tokenizer

USER_MAXLEN=17
tokenizer = Tokenizer(user_maxlen=USER_MAXLEN)
tokenizer.load_dictionaries(path)

## D) Testing 

In [8]:
TMP_VIDEO_FILENAME = 'tmp.mp4'

def unpreprocess(video):
    video = video.data.numpy()[0]
    video = 64. * video.transpose(1, 2, 3, 0)
    return np.array(video, 'uint8')

def demo(net, preprocessor, filename, top_n=5):
    # Open mpeg file and get a numpy array
    video_uint8 = open_mpeg_video(filename, 12, [128, 128])
    # Preprocessing
    video = preprocessor(video_uint8)
    # Convert to torch variable
    video = Variable(torch.from_numpy(video[None]), volatile=True).cuda()
    empty_caption = Variable(torch.zeros([1, 1]), volatile=True).long().cuda()
    
    # Compute predictions
    pred, class_pred = net.forward((video, empty_caption), use_teacher_forcing=False)
    # Convert to numpy 
    pred = np.exp(pred.cpu().data.numpy())[0]
        
    pred_argmax = np.argmax(pred, axis=1)
    decoded_pred = tokenizer.decode_caption(pred_argmax)
    beautiful_caption = " ".join(str(e+" ") for e in decoded_pred if "<END>" not in e)
    #print('__CAPTION__: {}'.format(beautiful_caption))
    
    
    # Class index
    class_index = torch.max(class_pred, dim=1)[1].cpu().data[0]
    cls = int2label[class_index]
    #print('ACTION: {:60s}\n'.format(cls))
    
    
    matched_action = get_template(decoded_pred, templates, tokenizer)
    # print(actions)
    objects = get_object_tokens(decoded_pred, matched_action[0][0])

    
    objects_list = extract_objects(objects)
    # Print class name with proba
    # Save input video in tmp file
    vwrite(TMP_VIDEO_FILENAME, unpreprocess(video.cpu()))
    return beautiful_caption, cls, objects_list, matched_action[0][0]


def path_generator(annotation_path, root_path):
    with gzip.open(annotation_path, "rt") as f:
        annotations = json.load(f)
    files = [elem['file'] for elem in annotations]
    labels = [elem['label'] for elem in annotations]
    placeholders = [elem['placeholders'] for elem in annotations] 
    actions = [elem['template'].replace("[","").replace("]", "").replace(",","").upper() for elem in annotations]
    
    return ((os.path.join(root_path, f), label, a, p) for f,label,p, a in zip(files, labels, placeholders, actions))

In [9]:
# Path generator
path_gen = path_generator('/data/20bn-somethingsomething/json/test_20170929.json.gz', 
                          '/data/20bn-somethingsomething/videos')
# Put the netwoark in evaluation mode
_ = net.eval()

#### Preprocessor

In [10]:
from rtorchn.data.preprocessing import default_evaluation_preprocesser

preprocessor = default_evaluation_preprocesser([48, 96, 96], 64.)

In [11]:
for i in range(5000):
    path_to_video, label, _,_ = next(path_gen)

## Longest Common Subsequence

In [12]:
def get_template(candidates, templates, tokenizer):
   
    lcs = LCS([fscore], tokenizer)
    
    max_templates = []
    #print("There are {} templates".format(len(templates)))

    candidates = [" ".join(candidates)]
    for candidate in candidates:
        
        max_lcs_template = ""
        max_lcs_value = -1
        for template in templates:
            lcs_value = compute_LCS(lcs, candidate, template, tokenizer)
            if lcs_value > max_lcs_value:
                max_lcs_template = template
                max_lcs_value = lcs_value
        max_templates.append((max_lcs_template, max_lcs_value))
#         print("Candidate: {}".format(candidate))
        #print("MATCHED ACTION : {}".format(max_lcs_template))

    return max_templates


def compute_LCS(lcs, candidate, template, tokenizer):
    encoded_caption = Variable(
        torch.LongTensor([tokenizer.encode_caption(candidate)]))
    encoded_prediction = Variable(
        torch.LongTensor([tokenizer.encode_caption(template)]))
    score_attr = namedtuple("ScoresAttr", "string_captions captions predictions")
    in_tuple = score_attr([candidate], encoded_caption, encoded_prediction)
    lcs_output = lcs(in_tuple)
    return lcs_output['fscore']

def extract_objects(object_tokens_list):
    
    objects_list = []
    if len(object_tokens_list) == 0:
        return objects_list
    
    next_token_ind =  object_tokens_list[0][0]
    current_object = ""
    for  (ind, token) in object_tokens_list:
        if  next_token_ind == ind:
            current_object += token+" "
        else:
            objects_list.append(current_object+" ")
            current_object = token
            next_token_ind = ind
        next_token_ind += 1
        
        
    if len(current_object)>0:
        objects_list.append(current_object)
               
    #print("PREDICTED OBJECTS: {}".format(objects_list))
    return objects_list



def get_object_tokens(caption, template):
    return [(i,token) for (i,token) in enumerate(caption) if token not in template and token not in stop_words]

In [13]:
articles= ["the", "a", "an", "A", "An", "The"]

annotations = JsonParser.open_annotation("/data/20bn-somethingsomething/json/train_20171031.json.gz")
templates = np.unique(annotations["template"]) # A list of templates
objects = annotations["placeholders"]
obj_tokens = [token for token in objects if token not in stop_words]
all_obj=[item for sublist in objects for item in sublist]
filtered_obj =  [" ".join(obj) for obj in all_obj for  token in obj if token not in articles]
templates = [" ".join(tokenizer.tokenize(t)) for t in templates]


# get_objects(sentences1[0].split(), sentence1_templates[0].split())



In [14]:
articles= ["the", "a", "an", "A", "An", "The"]

fil3 = [token.upper() for obj in all_obj for token in obj.split(" ")]
fil2 = [[token for token in obj.split(" ") if token not in articles ] for obj in all_obj]
fil = list( map(lambda p:" ".join([token for token in p.split(" ") if token not in articles ]), all_obj))



In [17]:
all_objects = {}
correct_objects = {}
all_actions = {}
correct_actions = {}

for i in range(1000):
    path_to_video, target_caption, target_action, target_objects = next(path_gen)
    for p in target_objects:
        p_tokens = p.split(" ")
        for pto in p_tokens:
            all_objects[pto.strip().upper()] = all_objects.get(pto.strip().upper(), 0) + 1
        
    all_actions[target_action] = all_actions.get(target_action, 0) + 1

   
    print('sample {}'.format(i)) 
    pred_caption, pred_action, pred_objects, matched_action = demo(net, preprocessor, path_to_video)
    for (i,o) in enumerate(pred_objects):
        o_tokens = o.strip().split(" ")
        for oto in o_tokens:
            if i<len(target_objects) and oto in target_objects[i].upper():
                correct_objects[oto] = correct_objects.get(oto, 0) + 1
                print("woohoo")
               
          
    print('TARGET CAPTION: {}'.format(target_caption))
    print('PRED   CAPTION: {}\n'.format(pred_caption))
    print('TARGET  ACTION: {}'.format(target_action))
    print('CLASSIF ACTION: {}'.format(pred_action))
    print('CAPTION ACTION: {}\n'.format(matched_action))
            
    print('TARGET OBJECTS: {}'.format(target_objects))
    print('PRED   OBJECTS: {}\n'.format(pred_objects))


    if matched_action == target_action:
        print("yesss")
        correct_actions[matched_action] = correct_actions.get(matched_action, 0) + 1
        
    print('{}\n'.format('-'*65))
    
print(all_objects)
print(correct_objects)

print(all_actions)
print(correct_actions)
    

sample 0
woohoo
TARGET CAPTION: tearing tissue paper just a little bit
PRED   CAPTION: TEARING  PAPER  JUST  A  LITTLE  BIT 

TARGET  ACTION: TEARING SOMETHING JUST A LITTLE BIT
CLASSIF ACTION: Tearing [something] just a little bit
CAPTION ACTION: TEARING SOMETHING JUST A LITTLE BIT

TARGET OBJECTS: ['tissue paper']
PRED   OBJECTS: ['PAPER ']

yesss
-----------------------------------------------------------------

sample 1
TARGET CAPTION: taking scissor out of box
PRED   CAPTION: TAKING  <UNK>  FROM  <UNK> 

TARGET  ACTION: TAKING SOMETHING OUT OF SOMETHING
CLASSIF ACTION: Taking [something] from [somewhere]
CAPTION ACTION: TAKING SOMETHING FROM SOMEWHERE

TARGET OBJECTS: ['scissor', 'box']
PRED   OBJECTS: ['<UNK>  ', '<UNK>']

-----------------------------------------------------------------

sample 2
woohoo
TARGET CAPTION: lifting book with bat on it
PRED   CAPTION: LIFTING  A  BOOK  WITH  A  PEN  ON  IT 

TARGET  ACTION: LIFTING SOMETHING WITH SOMETHING ON IT
CLASSIF ACTION: Taking

woohoo
TARGET CAPTION: pouring water onto wood
PRED   CAPTION: SPILLING  WATER  NEXT  TO  A  PEN 

TARGET  ACTION: POURING SOMETHING ONTO SOMETHING
CLASSIF ACTION: Spilling [something] next to [something]
CAPTION ACTION: SPILLING SOMETHING NEXT TO SOMETHING

TARGET OBJECTS: ['water', 'wood']
PRED   OBJECTS: ['WATER  ', 'PEN']

-----------------------------------------------------------------

sample 21
TARGET CAPTION: moving scissors up
PRED   CAPTION: MOVING  A  MOBILE  PHONE  UP 

TARGET  ACTION: MOVING SOMETHING UP
CLASSIF ACTION: Moving [something] up
CAPTION ACTION: MOVING SOMETHING UP

TARGET OBJECTS: ['scissors']
PRED   OBJECTS: ['MOBILE PHONE ']

yesss
-----------------------------------------------------------------

sample 22
TARGET CAPTION: putting toy on a surface
PRED   CAPTION: PUTTING  A  PENNY  ON  A  SURFACE 

TARGET  ACTION: PUTTING SOMETHING ON A SURFACE
CLASSIF ACTION: Putting [something] on a surface
CAPTION ACTION: PUTTING SOMETHING ON A SURFACE

TARGET OBJECTS: [

TARGET CAPTION: pretending to take Chocolate  from Box
PRED   CAPTION: PUTTING  A  <UNK> 

TARGET  ACTION: PRETENDING TO TAKE SOMETHING FROM SOMEWHERE
CLASSIF ACTION: Attaching [something] to [something]
CAPTION ACTION: PUTTING SOMETHING ON A SURFACE

TARGET OBJECTS: ['Chocolate ', 'Box']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 41
TARGET CAPTION: holding Remote behind Mobile
PRED   CAPTION: HOLDING  A  PEN  BEHIND  THE  POUCH 

TARGET  ACTION: HOLDING SOMETHING BEHIND SOMETHING
CLASSIF ACTION: Holding [something] behind [something]
CAPTION ACTION: HOLDING SOMETHING BEHIND SOMETHING

TARGET OBJECTS: ['Remote', 'Mobile']
PRED   OBJECTS: ['PEN  ', 'POUCH']

yesss
-----------------------------------------------------------------

sample 42
TARGET CAPTION: squeezing a stuffed animal
PRED   CAPTION: SQUEEZING  A  SPONGE 

TARGET  ACTION: SQUEEZING SOMETHING
CLASSIF ACTION: Squeezing [something]
CAPTION ACTION: SQUEEZING SOMETHING


TARGET CAPTION: holding A stick
PRED   CAPTION: BENDING  A  PLASTIC  SPOON  UNTIL  IT  BREAKS 

TARGET  ACTION: HOLDING SOMETHING
CLASSIF ACTION: Putting [something that cannot actually stand upright] upright on the table, so it falls on its side
CAPTION ACTION: BENDING SOMETHING UNTIL IT BREAKS

TARGET OBJECTS: ['A stick']
PRED   OBJECTS: ['PLASTIC SPOON ']

-----------------------------------------------------------------

sample 61
woohoo
TARGET CAPTION: showing a glas bottle behind a wire
PRED   CAPTION: SHOWING  A  BOTTLE  BEHIND  A  BOTTLE 

TARGET  ACTION: SHOWING SOMETHING BEHIND SOMETHING
CLASSIF ACTION: Showing [something] behind [something]
CAPTION ACTION: SHOWING SOMETHING BEHIND SOMETHING

TARGET OBJECTS: ['a glas bottle', 'a wire']
PRED   OBJECTS: ['BOTTLE  ', 'BOTTLE']

yesss
-----------------------------------------------------------------

sample 62
TARGET CAPTION: hitting Chair with Notebook 
PRED   CAPTION: HITTING  A  PILLOW  WITH  A  PILLOW 

TARGET  ACTION: HITTIN

TARGET CAPTION: tearing paper towel  into two pieces
PRED   CAPTION: MOVING  A  BOTTLE  AND  A  BOTTLE  AWAY  FROM  EACH  OTHER 

TARGET  ACTION: TEARING SOMETHING INTO TWO PIECES
CLASSIF ACTION: Moving [something] and [something] away from each other
CAPTION ACTION: MOVING SOMETHING AND SOMETHING AWAY FROM EACH OTHER

TARGET OBJECTS: ['paper towel ']
PRED   OBJECTS: ['BOTTLE  ', 'BOTTLE']

-----------------------------------------------------------------

sample 82
TARGET CAPTION: squeezing purse
PRED   CAPTION: SQUEEZING  A  WATER  BOTTLE 

TARGET  ACTION: SQUEEZING SOMETHING
CLASSIF ACTION: Throwing [something] in the air and catching it
CAPTION ACTION: SQUEEZING SOMETHING

TARGET OBJECTS: ['purse']
PRED   OBJECTS: ['WATER BOTTLE ']

yesss
-----------------------------------------------------------------

sample 83
TARGET CAPTION: lifting wallet with hair clip on it
PRED   CAPTION: LIFTING  BOOK  WITH  <UNK>  ON  IT 

TARGET  ACTION: LIFTING SOMETHING WITH SOMETHING ON IT
CLASSIF AC

...

woohoo
TARGET CAPTION: pouring Water into Toilet
PRED   CAPTION: POURING  WATER  INTO  CUP 

TARGET  ACTION: POURING SOMETHING INTO SOMETHING
CLASSIF ACTION: Pouring [something] into [something]
CAPTION ACTION: POURING SOMETHING INTO SOMETHING

TARGET OBJECTS: ['Water', 'Toilet']
PRED   OBJECTS: ['WATER  ', 'CUP']

yesss
-----------------------------------------------------------------

sample 905
TARGET CAPTION: plugging hair straightener into wall outlet
PRED   CAPTION: TWISTING  WRINGING  A  SPONGE  WET  UNTIL  WATER  COMES  OUT 

TARGET  ACTION: PLUGGING SOMETHING INTO SOMETHING
CLASSIF ACTION: Twisting [something]
CAPTION ACTION: TWISTING WRINGING SOMETHING WET UNTIL WATER COMES OUT

TARGET OBJECTS: ['hair straightener', 'wall outlet']
PRED   OBJECTS: ['SPONGE ']

-----------------------------------------------------------------

sample 906
TARGET CAPTION: twisting twisting a cap to install
PRED   CAPTION: TEARING  PAPER  INTO  TWO  PIECES 

TARGET  ACTION: TWISTING SOMETHING
CLAS

TARGET CAPTION: taking one of many fortune slips on the table
PRED   CAPTION: TAKING  ONE  OF  MANY  SIMILAR  SWEETS 

TARGET  ACTION: TAKING ONE OF MANY SIMILAR THINGS ON THE TABLE
CLASSIF ACTION: Taking [one of many similar things on the table]
CAPTION ACTION: TAKING ONE OF MANY SIMILAR THINGS ON THE TABLE

TARGET OBJECTS: ['one of many fortune slips on the table']
PRED   OBJECTS: ['SWEETS ']

yesss
-----------------------------------------------------------------

sample 925
TARGET CAPTION: letting a ball roll along a flat surface
PRED   CAPTION: LETTING  A  SPRAY  BOTTLE  ROLL  ALONG  A  FLAT  SURFACE 

TARGET  ACTION: LETTING SOMETHING ROLL ALONG A FLAT SURFACE
CLASSIF ACTION: Letting [something] roll along a flat surface
CAPTION ACTION: LETTING SOMETHING ROLL ALONG A FLAT SURFACE

TARGET OBJECTS: ['a ball']
PRED   OBJECTS: ['SPRAY BOTTLE ']

yesss
-----------------------------------------------------------------

sample 926
TARGET CAPTION: showing Pillow behind Cushion
PRED   CAP

TARGET CAPTION: holding lipstick over butter knife
PRED   CAPTION: SHOWING  A  SHADOW  OF  THERMOS  MAKING  SURE  THE  SHADOW  IS  NOT  MOVING 

TARGET  ACTION: HOLDING SOMETHING OVER SOMETHING
CLASSIF ACTION: Holding [something] over [something]
CAPTION ACTION: SHOWING A SHADOW OF SOMETHING MAKING SURE THE SHADOW IS NOT MOVING

TARGET OBJECTS: ['lipstick', 'butter knife']
PRED   OBJECTS: ['THERMOS ']

-----------------------------------------------------------------

sample 945
TARGET CAPTION: tearing a napkin into two pieces
PRED   CAPTION: TEARING  PAPER  INTO  TWO  PIECES 

TARGET  ACTION: TEARING SOMETHING INTO TWO PIECES
CLASSIF ACTION: Tearing [something] into two pieces
CAPTION ACTION: TEARING SOMETHING INTO TWO PIECES

TARGET OBJECTS: ['a napkin']
PRED   OBJECTS: ['PAPER ']

yesss
-----------------------------------------------------------------

sample 946
woohoo
TARGET CAPTION: pouring water out of cup
PRED   CAPTION: POURING  WATER  OUT  OF  PLASTIC  CUP 

TARGET  ACTION: P

woohoo
woohoo
TARGET CAPTION: pouring Water into Cup
PRED   CAPTION: POURING  WATER  INTO  A  CUP 

TARGET  ACTION: POURING SOMETHING INTO SOMETHING
CLASSIF ACTION: Pouring [something] into [something]
CAPTION ACTION: POURING SOMETHING INTO SOMETHING

TARGET OBJECTS: ['Water', 'Cup']
PRED   OBJECTS: ['WATER  ', 'CUP']

yesss
-----------------------------------------------------------------

sample 964
TARGET CAPTION: turning the camera right while filming small bottle
PRED   CAPTION: TURNING  THE  CAMERA  RIGHT  WHILE  FILMING  PENDRIVE 

TARGET  ACTION: TURNING THE CAMERA RIGHT WHILE FILMING SOMETHING
CLASSIF ACTION: Turning the camera right while filming [something]
CAPTION ACTION: TURNING THE CAMERA RIGHT WHILE FILMING SOMETHING

TARGET OBJECTS: ['small bottle']
PRED   OBJECTS: ['PENDRIVE ']

yesss
-----------------------------------------------------------------

sample 965
woohoo
TARGET CAPTION: putting tv remote next to pill bottle
PRED   CAPTION: MOVING  A  CUP  CLOSER  TO  A  B

TARGET CAPTION: dropping Ping pong ball onto Floor
PRED   CAPTION: DROPPING  A  PEN  ONTO  A  TABLE 

TARGET  ACTION: DROPPING SOMETHING ONTO SOMETHING
CLASSIF ACTION: [Something] falling like a rock
CAPTION ACTION: DROPPING SOMETHING ONTO SOMETHING

TARGET OBJECTS: ['Ping pong ball', 'Floor']
PRED   OBJECTS: ['PEN  ', 'TABLE']

yesss
-----------------------------------------------------------------

sample 984
TARGET CAPTION: showing a shadow of a deodorant, making sure the shadow is not moving.
PRED   CAPTION: SHOWING  A  BOTTLE  BEHIND  A  GLASS 

TARGET  ACTION: SHOWING A SHADOW OF SOMETHING MAKING SURE THE SHADOW IS NOT MOVING.
CLASSIF ACTION: Showing [something] behind [something]
CAPTION ACTION: SHOWING SOMETHING BEHIND SOMETHING

TARGET OBJECTS: ['a deodorant']
PRED   OBJECTS: ['BOTTLE  ', 'GLASS']

-----------------------------------------------------------------

sample 985
TARGET CAPTION: a deodorant falling like a rock
PRED   CAPTION: DROPPING  A  BALL  ONTO  A  CHAIR 

TAR

In [ ]:
for ca in correct_actions:
    
    print("{}/{} of {} actions correct".format(correct_actions[ca], all_actions[ca], ca))

In [26]:
for correct_key in correct_objects.keys():
    denom = 0
    if correct_key in all_objects.keys():
        denom += all_objects[correct_key]
        #print("{}:{}".format(correct_key, all_objects[correct_key]))
    #for j in all_objects.keys():
    #    if correct_key in j or correct_key in j:
    #        print("{}:{}".format(j, all_objects[j]))
    #        denom += all_objects[j]
        
                    
    print (">>model got  {}/{} of '{}'s correct".format(correct_objects[correct_key],denom, correct_key ))
    print("-"*100)
    

>>model got  3/7 of 'SOCKET's correct
----------------------------------------------------------------------------------------------------
>>model got  6/63 of 'BOX's correct
----------------------------------------------------------------------------------------------------
>>model got  1/5 of 'SPINNER's correct
----------------------------------------------------------------------------------------------------
>>model got  21/38 of 'WATER's correct
----------------------------------------------------------------------------------------------------
>>model got  3/9 of 'LAPTOP's correct
----------------------------------------------------------------------------------------------------
>>model got  1/1 of 'SHOES's correct
----------------------------------------------------------------------------------------------------
>>model got  7/42 of 'CUP's correct
----------------------------------------------------------------------------------------------------
>>model got  1/5 of 'KEYBOARD'

In [18]:
a = sum(correct_objects.values())
b = sum(all_objects.values())

print("{} out of {} objects are correctly predicted: {:.2}% ".format(a, b, a/b*100))

164 out of 2341 objects are correctly predicted: 7.0% 


In [19]:
len(all_objects.keys())

595

In [20]:
c = sum(correct_actions.values())
d = sum(all_actions.values())

print("{} out of {} actions are correctly predicted: {:}% ".format(c, d, c/d*100))

372 out of 1000 actions are correctly predicted: 37.2% 


In [32]:
len(all_actions.keys())

164